In [1]:
# !pip install beir

In [2]:
# !pip install pytorch-lightning==1.9.5

In [3]:
# !pip install beir

In [4]:
# !pip install -e ../crowd-coachable-recommendations/

In [5]:
# !pip install --no-cache-dir --ignore-installed -U numba

In [ ]:
# !(cd ../crowd-coachable-recommendations/data/amazon_review_prime_pantry && wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz)

In [6]:
import os
os.environ['CCREC_MAX_LENGTH'] = '256'
os.environ['CCREC_SIM_TYPE'] = 'dot'
os.environ['CCREC_EMBEDDING_TYPE'] = 'mean_pooling'
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

In [7]:
import pandas as pd

In [8]:
from collections import Counter

In [9]:
frac = None

In [10]:
%run -n ../crowd-coachable-recommendations/scripts/al_oracle_agent.py

initializing env defaults
using   CCREC_EMBEDDING_TYPE=mean_pooling; options: ['cls', 'mu', 'mean', 'mean_pooling', 'mean_layer_norm']
using   CCREC_MAX_LENGTH=256; options: None
using   CCREC_SIM_TYPE=dot; options: ['cos', 'dot']
setting CCREC_TRAIN_MAIN=bmt_main; options: ['bmt_main', 'bbpr_main']
setting CCREC_LEGACY_VAE_BUG=0; options: ['0', '1']
setting CCREC_DEBUG_SHAP=0; options: ['0', '1']
setting CCREC_TRAINING_PRECISION=32; options: ['32', 'bf16']
setting CCREC_BBPR_INV_TEMPERATURE=20; options: None
setting CCREC_DISPLAY_LENGTH=250; options: None
setting CCREC_INIT_ENV_DONE=1; options: ['0', '1']
setting CCREC_NON_BLOCKING=1; options: ['0', '1']


/home/ec2-user/efs/shared_folder/crowd-coachable-recommendations/src/ccrec/__init__.py:44: UserWarning: dot similarity works best with small inv_temperature
  warnings.warn("dot similarity works best with small inv_temperature")
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for thi

Using device: cuda


The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.


In [11]:
DATA_NAME = "nq"
SUFFIX='-contriever-dot'
N_STEPS=1
MODEL_NAME='facebook/contriever'

In [12]:
save_root = "{}_results_oracle_agent_{}{}".format(DATA_NAME, ACCURACY_LEVEL, SUFFIX)
print(save_root)

nq_results_oracle_agent_1.0-contriever-dot


# corpus, queries, qrels

In [13]:
corpus, queries, qrels, *extra = load_data(DATA_NAME)

  0%|          | 0/2681468 [00:00<?, ?it/s]

In [14]:
if len(extra):
    block_dict, qids_split, item_df = extra
    number_of_qid_split_batch = len(qids_split)
else:  # standard qids_split creation, though we used previous splits for stability
    block_dict = None
    number_of_qid_split_batch = 4
    qids_split = np.array_split(
        np.random.RandomState(42).permutation(list(queries.keys())),
        number_of_qid_split_batch
    )
    qids_split = [s.tolist() for s in qids_split]

# bm25

In [ ]:
from bm_25 import BM25
import tqdm

bm25_agent = BM25(b=0.75, k1=1.2).fit(corpus.values())
corpus_keys = list(corpus.keys())

In [29]:
bm25_profile = {}
for qid in tqdm.tqdm(queries.keys()):
    scores = bm25_agent.transform(queries[qid])
    if block_dict is not None:
        scores[
            item_df.index.get_indexer(block_dict[qid])
        ] = -1e6
    top100 = torch.as_tensor(scores).topk(100).indices.numpy()
    bm25_profile[qid] = {
        corpus_keys[a]: scores[a]
        for a in top100
    }

100%|██████████| 3452/3452 [21:19<00:00,  2.70it/s]


In [30]:
evaluator = EvaluateRetrieval(None)
evaluator.evaluate_custom(
    qrels, bm25_profile, [1, 5, 10, 100], metric="mrr"
)

{'MRR@1': 0.1489, 'MRR@5': 0.23039, 'MRR@10': 0.24446, 'MRR@100': 0.25604}

In [31]:
bm25_path = f"{save_root}/data_iteration_0/ranking_profile_bm25.pt"

In [32]:
!mkdir -p {os.path.dirname(bm25_path)}

In [33]:
torch.save(bm25_profile, bm25_path)

# start

In [34]:
print("Accuracy level:", ACCURACY_LEVEL)
print("Total number of iterations:", number_of_qid_split_batch * N_STEPS)

ranking_profile_bm25 = torch.load(bm25_path)

# Load initial model
model_init_dir = None
model = _BertBPR(None, model_name=MODEL_NAME)

Accuracy level: 1.0
Total number of iterations: 4


In [35]:
for step in range(N_STEPS * number_of_qid_split_batch + 1):
    previous_working_dir = f"{save_root}/data_iteration_{step-1}"
    current_working_dir = f"{save_root}/data_iteration_{step}"

    if not os.path.exists(current_working_dir):
        os.makedirs(current_working_dir)

    # generate ranking profile
    print("Generate ranking profile at step:", step)
    save_path = os.path.join(current_working_dir, "ranking_profile.pt")
    if os.path.isfile(save_path):
        ranking_profile = torch.load(save_path)
    else:
        with autocast():
            ranking_profile = generate_ranking_profile(
                model, MODEL_NAME, corpus, queries, qrels, save_path, block_dict)

    evaluator = EvaluateRetrieval(None)
    mrr = evaluator.evaluate_custom(
        qrels, ranking_profile, [1, 5, 10, 100], metric="mrr"
    )
    for name, value in mrr.items():
        print("{}".format(name), ":", value)

    if step >= N_STEPS * number_of_qid_split_batch:
        break  # moved here after running

    # generate training data
    print("Generate training data at step:", step)

    train_data = generate_train_data(
        qids_split[step % number_of_qid_split_batch],
        qrels,
        ranking_profile,
        ranking_profile_bm25,
        list(corpus.keys()),
        step,
    )

    if step > 0:
        train_data_prev_dir = os.path.join(
            previous_working_dir, "training_data.pt"
        )
        train_data_prev = torch.load(train_data_prev_dir)
        train_data = combine_train_data(train_data_prev, train_data)

    train_data_dir = os.path.join(current_working_dir, "training_data.pt")
    torch.save(train_data, train_data_dir)
    
    # train model
    print("Training model at step:", step)
    print("Number of training data:", len(train_data))
    model = training(
        train_data,
        NUM_EPOCHS,
        model_checkpoint=model_init_dir,
        corpus=corpus,
        queries=queries,
        model_selection=MODEL_NAME,
    )

    torch.save(
        model.model.item_tower.state_dict(),
        f'{current_working_dir}/state-dict.pth',
    )

Generate ranking profile at step: 0
caching replicas
Processed 2048 | 3452 t=2.5s / 4.3s
Processed total 3452 t=3.2s
Processed 2048 | 2681468 t=0.8s / 1024.5s
Processed 4096 | 2681468 t=1.6s / 1027.8s
Processed 8192 | 2681468 t=3.1s / 1023.2s
Processed 16384 | 2681468 t=6.3s / 1033.3s
Processed 32768 | 2681468 t=13.3s / 1087.5s
Processed 65536 | 2681468 t=27.9s / 1143.1s
Processed 131072 | 2681468 t=56.8s / 1162.8s
Processed 262144 | 2681468 t=114.9s / 1175.6s
Processed 524288 | 2681468 t=231.0s / 1181.6s
Processed 1048576 | 2681468 t=463.3s / 1184.7s
Processed 2097152 | 2681468 t=930.7s / 1190.0s
Processed total 2681468 t=1205.2s
MRR@1 : 0.14455
MRR@5 : 0.22817
MRR@10 : 0.24384
MRR@100 : 0.25769
MRR@1 : 0.14455
MRR@5 : 0.22817
MRR@10 : 0.24384
MRR@100 : 0.25769
Generate training data at step: 0
Training model at step: 0
Number of training data: 274
timing checking for repeated user-item events done . time 0.0s
timing inferring training events done . time 0.0s
timing creating target_cs

Setting `Trainer(gpus=4)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=4)` instead.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


multiple_nrl alpha=1.0
masked False
ct_num_batches 45.233333333333334 ft_num_batches 9.133333333333333 ct_cycles 1 ft_cycles 4.952554744525548
train_set size 274


Map:   0%|          | 0/1357 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4,5,6,7]

  | Name       | Type           | Params
----------------------------------------------
0 | item_tower | NaiveItemTower | 109 M 
----------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.935   Total estimated model params size (MB)


...

Sanity Checking: 0it [00:00, ?it/s]

One of given dataloaders is None and it will be skipped.


Training: 0it [00:00, ?it/s]

Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 28. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
`Trainer.fit` stopped: `max_epochs=10` reached.
No checkpoints found!


model.load_state_dict(torch.load(...), strict=False)
/home/ec2-user/efs/shared_folder/notebook_prime_pantry/lightning_logs/version_32/checkpoints/state-dict.pth
fit time 1522.1s
timing checking for repeated user-item events done . time 0.0s
timing inferring training events done . time 0.0s
timing creating target_csr timing joining testing events by multi-indexed requests done . time 0.0s
done . time 0.0s
timing creating prior_score done . time 0.0s
timing creating reranking tasks by adding prior_score from target candidates done . time 0.0s
Dataset(1498, 7401) with 1498 target events and 7427 prior scores created!
masked False
ct_num_batches 246.7 ft_num_batches 49.93333333333333 ct_cycles 1 ft_cycles 4.940587449933244
Processing 0 | 8
transform time 10.1s
_assign_topk time 0.1s
evaluate_assigned time 0.0s
Generate ranking profile at step: 4
caching replicas
Processed 2048 | 3452 t=0.2s / 0.4s
Processed total 3452 t=0.4s
Processed 2048 | 2681468 t=0.8s / 1052.3s
Processed 4096 | 268146

In [36]:
def print_all_mrr():
    for step in range(number_of_qid_split_batch+1):
        current_working_dir = f"{save_root}/data_iteration_{step}"
        ranking_profile = torch.load(current_working_dir + '/ranking_profile.pt')
        evaluator = EvaluateRetrieval(None)
        mrr = evaluator.evaluate_custom(
            qrels, ranking_profile, [1, 5, 10, 100], metric="mrr"
        )
        for name, value in mrr.items():
            print("{}".format(name), ":", value)

In [37]:
print_all_mrr()

MRR@1 : 0.14455
MRR@5 : 0.22817
MRR@10 : 0.24384
MRR@100 : 0.25769
MRR@1 : 0.29838
MRR@5 : 0.39001
MRR@10 : 0.40534
MRR@100 : 0.41558
MRR@1 : 0.36935
MRR@5 : 0.44443
MRR@10 : 0.45795
MRR@100 : 0.46755
MRR@1 : 0.41425
MRR@5 : 0.4832
MRR@10 : 0.49471
MRR@100 : 0.50381
MRR@1 : 0.47827
MRR@5 : 0.53532
MRR@10 : 0.54807
MRR@100 : 0.55574


In [40]:
def print_gen_mrr():
    for step in range(number_of_qid_split_batch+1):
        current_working_dir = f"{save_root}/data_iteration_{step}"
        ranking_profile = torch.load(current_working_dir + '/ranking_profile.pt')
        evaluator = EvaluateRetrieval(None)
        mrr = evaluator.evaluate_custom(
            {k:v for k,v in qrels.items() if k in qids_split[-1]},
            {k:v for k,v in ranking_profile.items() if k in qids_split[-1]},
            [1, 5, 10, 100], metric="mrr"
        )
        for name, value in mrr.items():
            print("{}".format(name), ":", value)

In [41]:
print_gen_mrr()

MRR@1 : 0.13905
MRR@5 : 0.22578
MRR@10 : 0.24181
MRR@100 : 0.25584
MRR@1 : 0.26999
MRR@5 : 0.37466
MRR@10 : 0.39031
MRR@100 : 0.40062
MRR@1 : 0.30012
MRR@5 : 0.3995
MRR@10 : 0.41412
MRR@100 : 0.4241
MRR@1 : 0.30591
MRR@5 : 0.42016
MRR@10 : 0.43279
MRR@100 : 0.4423
MRR@1 : 0.5168
MRR@5 : 0.56419
MRR@10 : 0.57684
MRR@100 : 0.58492
